<a href="https://colab.research.google.com/github/maneet2004/Exoplanet-Classification-using-Spiking-Neural-Networks/blob/main/Exoplanet_classification_using_SNNtorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Reshape, Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers.schedules import ExponentialDecay

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


from keras.optimizers import Adam
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.optimizers.schedules import ExponentialDecay
from keras.models import load_model
from itertools import chain

In [ ]:
exoTrain = pd.read_csv('/content/sample_data/exoTrain.csv')
exoTest = pd.read_csv('/content/sample_data/exoTest.csv')

exoTrain.tail(5)

In [ ]:
print(exoTrain['LABEL'].value_counts())
print(exoTest['LABEL'].value_counts())

In [ ]:
import matplotlib.pyplot as plt

# Plot a specific FLUX column
plt.plot(df['FLUX.1'])
plt.title('FLUX.1 Over Time')
plt.xlabel('Index')
plt.ylabel('FLUX.1')
plt.show()

In [ ]:
print(exoTrain['LABEL'].value_counts())
print(exoTest['LABEL'].value_counts())

In [ ]:
def flux_graph(dataset, row, dataframe, planet):
    if dataframe:
        fig = plt.figure(figsize=(20,5))
        ax = fig.add_subplot()
        ax.set_title(planet, color='black', fontsize=22)
        ax.set_xlabel('time', color='black', fontsize=18)
        ax.set_ylabel('flux_' + str(row), color='black', fontsize=18)
        ax.grid(False)
        flux_time = list(dataset.columns)
        flux_values = dataset[flux_time].iloc[row]
        ax.plot([i + 1 for i in range(dataset.shape[1])], flux_values, 'black')
        ax.tick_params(colors = 'black', labelcolor='black', labelsize=14)
        plt.show()
    else:
        fig = plt.figure(figsize=(20,5))
        ax = fig.add_subplot()

        ax.set_title(planet, color='black', fontsize=22)
        ax.set_xlabel('time', color='black', fontsize=18)
        ax.set_ylabel('flux_' + str(row), color='white', fontsize=18)
        ax.grid(False)
        flux_values = dataset[row]
        ax.plot([i + 1 for i in range(dataset.shape[1])], flux_values, 'black')
        ax.tick_params(colors = 'black', labelcolor='black', labelsize=14)
        plt.show()

In [ ]:
def show_graph(dataframe, dataset):
    with_planet = exoTrain[exoTrain['LABEL'] == 2].head(3).index
    wo_planet = exoTrain[exoTrain['LABEL'] == 1].head(3).index

    def plot_flux_graph(indices, label):
        for row in indices:
            flux_graph(dataset, row, dataframe, planet=label)

    plot_flux_graph(with_planet, 'Plot for transiting planet')
    plot_flux_graph(wo_planet, 'Plot for no transiting planet')


In [ ]:
dataset = exoTrain.drop(columns='LABEL')  # Preprocess dataset outside the function call
show_graph(dataframe=True, dataset=dataset)


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_dataset = scaler.fit_transform(exoTrain.drop(columns='LABEL'))

show_graph(is_dataframe=False, dataset=scaled_dataset)


In [ ]:
import numpy as np

def handle_outliers(dataset, num_iterations):
    """
    Replaces outlier values in the dataset with the mean value of the respective row.

    Parameters:
    - dataset: pd.DataFrame, the dataset to handle.
    - num_iterations: int, number of iterations to repeat the outlier handling process.

    Returns:
    - pd.DataFrame with outliers replaced by row means.
    """
    dataset_handled = dataset.copy()  # Avoid modifying the original dataset

    for _ in range(num_iterations):
        for index, row in dataset_handled.iterrows():
            row_values = row.values
            row_mean = row_values.mean()

            threshold = 1.5 * np.std(row_values)
            outlier_indices = np.where((row_values > row_mean + threshold) | (row_values < row_mean - threshold))

            # Replace outliers with the mean value
            dataset_handled.iloc[index, outlier_indices] = row_mean

    return dataset_handled


In [ ]:
cleaned_dataset = handle_outliers(exoTrain.drop(columns='LABEL'), threshold=2)
show_graph(is_dataframe=True, dataset=cleaned_dataset)


In [ ]:
def convert_labels(y_train, y_test):
    """
    Convert labels: 2 -> 1, 1 -> 0.

    Parameters:
    train_labels (pd.Series): Labels for the training set.
    test_labels (pd.Series): Labels for the test set.

    Returns:
    tuple: Converted training and test labels.
    """
    binary_label = lambda x: 1 if x == 2 else 0
    train_labels_01 = train_labels.apply(binary_label)
    test_labels_01 = test_labels.apply(binary_label)

    return train_labels_01, test_labels_01


In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

def apply_smote(x_train, y_train, over_strategy=0.2, under_strategy=0.3, random_state=17):
    """
    Apply SMOTE and undersampling to balance the dataset.

    Parameters:
    x_train (pd.DataFrame): Features for the training set.
    y_train (pd.Series): Labels for the training set.
    over_strategy (float or dict, optional): SMOTE sampling strategy.
    under_strategy (float or dict, optional): Random undersampling strategy.
    random_state (int, optional): Seed for reproducibility.

    Returns:
    tuple: Resampled training features and labels.
    """
    smote = SMOTE(sampling_strategy=over_strategy, random_state=random_state)
    undersample = RandomUnderSampler(sampling_strategy=under_strategy, random_state=random_state)
    pipeline = Pipeline(steps=[('smote', smote), ('undersample', undersample)])

    x_train_res, y_train_res = pipeline.fit_resample(x_train, y_train)
    return x_train_res, y_train_res

# Usage example
y_labels = exoTrain['LABEL']
print("Before SMOTE:", y_labels.value_counts())

x_resampled, y_resampled = apply_smote(handled_dataset, y_labels)
print("After SMOTE:", y_resampled.value_counts())


In [ ]:
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
import matplotlib.pyplot as plt

# Define training and testing datasets
def prepare_datasets(exoTrain, exoTest, outlier_threshold=2, smote_strategy=0.2, under_strategy=0.3, random_state=17):
    """
    Prepare and preprocess the training and testing datasets.

    Parameters:
    exoTrain (pd.DataFrame): The training dataset.
    exoTest (pd.DataFrame): The testing dataset.
    outlier_threshold (float, optional): Threshold for handling outliers.
    smote_strategy (float or dict, optional): SMOTE oversampling strategy.
    under_strategy (float or dict, optional): Random undersampling strategy.
    random_state (int, optional): Seed for reproducibility.

    Returns:
    tuple: Preprocessed training and testing features and labels, number of features.
    """

    x_train, y_train = exoTrain.drop(columns='LABEL'), exoTrain['LABEL']
    x_test, y_test = exoTest.drop(columns='LABEL'), exoTest['LABEL']

    x_train = handle_outliers(x_train, outlier_threshold)


    x_train, y_train = apply_smote(x_train, y_train, over_strategy=smote_strategy, under_strategy=under_strategy, random_state=random_state)

    y_train, y_test = convert_labels(y_train, y_test)

    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)

    n_features = x_train.shape[1]

    return x_train, y_train, x_test, y_test, n_features

# Function to graph accuracy and loss
def plot_training_history(history):
    """
    Plot the training and validation loss and accuracy.

    Parameters:
    history: Keras History object containing training history.
    """
    plt.figure(figsize=(12, 8))

    plt.subplot(2, 1, 1)
    plt.plot(history.history['loss'], label='Training Loss', color='blue')
    plt.plot(history.history['val_loss'], label='Validation Loss', color='orange')
    plt.title('Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(2, 1, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy', color='blue')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='orange')
    plt.title('Accuracy Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(y_test, y_pred, labels=[0, 1], cmap="BuGn", font_scale=1.4, figsize=(10, 7), annot_size=16):
    """
    Plot a confusion matrix using Seaborn's heatmap for better visualization.

    Parameters:
    y_test (pd.Series or np.array): True labels.
    y_pred (pd.Series or np.array): Predicted labels.
    labels (list, optional): The labels to display in the confusion matrix.
    cmap (str, optional): Colormap for the heatmap.
    font_scale (float, optional): Font scale for the annotations.
    figsize (tuple, optional): Size of the figure.
    annot_size (int, optional): Size of the annotations in the heatmap.

    Returns:
    np.array: The confusion matrix as a numpy array.
    """
    # Generate the confusion matrix
    matrix = confusion_matrix(y_test, y_pred, labels=labels)

    # Convert to DataFrame for easier labeling
    df_cm = pd.DataFrame(matrix, index=labels, columns=labels)
    df_cm.index.name = 'True Label'
    df_cm.columns.name = 'Predicted Label'

    # Plot the confusion matrix
    plt.figure(figsize=figsize)
    sn.set(font_scale=font_scale)
    sn.heatmap(df_cm, cmap=cmap, annot=True, annot_kws={"size": annot_size}, fmt="d")
    plt.show()

    return matrix


In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report

def train_naive_bayes(x_train, y_train, x_test, y_test, var_smoothing=1e-9):

    model = GaussianNB(var_smoothing=var_smoothing)
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)

    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, target_names=['Class 0', 'Class 1'])

    print(f'Naive Bayes Accuracy: {accuracy:.4f}')
    print('Classification Report:\n', report)

    return model, accuracy, report


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

def train_random_forest(x_train, y_train, x_test, y_test,
                        n_estimators=100, max_depth=10, min_samples_split=2,
                        min_samples_leaf=1, max_features='auto', bootstrap=True,
                        random_state=17):

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        bootstrap=bootstrap,
        random_state=random_state
    )
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)

    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, target_names=['Class 0', 'Class 1'])

    print(f'Random Forest Accuracy: {accuracy:.4f}')
    print('Classification Report:\n', report)

    return model, accuracy, report


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report

def train_gbm(x_train, y_train, x_test, y_test,
              n_estimators=100, learning_rate=0.1, max_depth=3,
              min_samples_split=2, min_samples_leaf=1,
              subsample=1.0, max_features=None,
              random_state=17):

    model = GradientBoostingClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        subsample=subsample,
        max_features=max_features,
        random_state=random_state
    )
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)

    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, target_names=['Class 0', 'Class 1'])

    print(f'GBM Accuracy: {accuracy:.4f}')
    print('Classification Report:\n', report)

    return model, accuracy, report


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Reshape, Conv1D, MaxPooling1D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers.schedules import ExponentialDecay

def cnn_model():

    x_train, y_train, x_test, y_test, n_features = datasets()
    x_train, y_train = shuffle(x_train, y_train)

    model = Sequential()
    model.add(Reshape((n_features, 1), input_shape=(n_features,)))

    # Convolutional Layers
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu', kernel_regularizer=l2(0.01), padding='same'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Dropout(0.3))

    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', kernel_regularizer=l2(0.01), padding='same'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Dropout(0.3))

    model.add(Conv1D(filters=128, kernel_size=3, activation='relu', kernel_regularizer=l2(0.01), padding='same'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Dropout(0.3))

    # Fully Connected Layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dense(1, activation='sigmoid'))

    print(model.summary())

    lr_schedule = ExponentialDecay(initial_learning_rate=1e-3, decay_steps=5000, decay_rate=0.9)
    optimizer = Adam(learning_rate=lr_schedule)

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Callbacks
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

    history = model.fit(
        x_train, y_train,
        validation_split=0.2,
        batch_size=64,
        epochs=50,
        callbacks=[early_stop, reduce_lr],
        verbose=2
    )

    _, train_acc = model.evaluate(x_train, y_train, verbose=2)
    _, test_acc = model.evaluate(x_test, y_test, verbose=2)
    print(f'Train: {train_acc:.3f}, Test: {test_acc:.3f}')

    y_class_pred = (model.predict(x_test) > 0.5).astype("int32")
    y_pred = model.predict(x_test)

    graph_acc(history)

    matrix = conf_matrix(y_test, y_class_pred)

    prediction_metrics(y_test, y_pred, y_class_pred, matrix)

    return model


In [ ]:
cnn_model1=cnn_model()

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Reshape, Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers.schedules import ExponentialDecay

def cnn_model_2d():

    x_train, y_train, x_test, y_test, n_features = datasets()
    x_train, y_train = shuffle(x_train, y_train)


    img_size = int(n_features ** 0.5)
    x_train = x_train.reshape(-1, img_size, img_size, 1)
    x_test = x_test.reshape(-1, img_size, img_size, 1)

    # Architecture
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same',
                     kernel_regularizer=l2(0.01), input_shape=(img_size, img_size, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
    model.add(Dropout(0.3))

    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same',
                     kernel_regularizer=l2(0.01)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
    model.add(Dropout(0.3))

    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same',
                     kernel_regularizer=l2(0.01)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
    model.add(Dropout(0.3))

    # Fully Connected Layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dense(1, activation='sigmoid'))

    print(model.summary())

    lr_schedule = ExponentialDecay(initial_learning_rate=1e-3, decay_steps=5000, decay_rate=0.9)
    optimizer = Adam(learning_rate=lr_schedule)

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

    history = model.fit(
        x_train, y_train,
        validation_split=0.2,
        batch_size=64,
        epochs=50,
        callbacks=[early_stop, reduce_lr],
        verbose=2
    )

    _, train_acc = model.evaluate(x_train, y_train, verbose=2)
    _, test_acc = model.evaluate(x_test, y_test, verbose=2)
    print(f'Train: {train_acc:.3f}, Test: {test_acc:.3f}')

    y_class_pred = (model.predict(x_test) > 0.5).astype("int32")
    y_pred = model.predict(x_test)

    graph_acc(history)

    matrix = conf_matrix(y_test, y_class_pred)

    prediction_metrics(y_test, y_pred, y_class_pred, matrix)

    return model


In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

def vgg16_model(input_shape, num_classes):

    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base_model.layers:
        layer.trainable = False

    model = Sequential()
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    lr_schedule = ExponentialDecay(initial_learning_rate=1e-3, decay_steps=10000, decay_rate=0.9)
    optimizer = Adam(learning_rate=lr_schedule)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

def prepare_data():

    x_train = np.random.rand(1000, 224, 224, 3)
    y_train = np.random.randint(0, 2, 1000)
    x_test = np.random.rand(200, 224, 224, 3)
    y_test = np.random.randint(0, 2, 200)

    y_train = to_categorical(y_train, num_classes=2)
    y_test = to_categorical(y_test, num_classes=2)

    return x_train, y_train, x_test, y_test

def train_vgg16_model():
    x_train, y_train, x_test, y_test = prepare_data()

    input_shape = (224, 224, 3)
    num_classes = 2
    model = vgg16_model(input_shape, num_classes)

    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

    history = model.fit(
        x_train, y_train,
        validation_split=0.2,
        batch_size=32,
        epochs=50,
        callbacks=[early_stop, reduce_lr],
        verbose=2
    )

    _, train_acc = model.evaluate(x_train, y_train, verbose=2)
    _, test_acc = model.evaluate(x_test, y_test, verbose=2)
    print(f'Train Accuracy: {train_acc:.3f}, Test Accuracy: {test_acc:.3f}')

    y_pred = model.predict(x_test)
    y_class_pred = np.argmax(y_pred, axis=1)
    y_test_labels = np.argmax(y_test, axis=1)


    graph_acc(history)
    matrix = conf_matrix(y_test_labels, y_class_pred)
    prediction_metrics(y_test_labels, y_pred, y_class_pred, matrix)

    return model

trained_model = train_vgg16_model()


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils import shuffle

def mlp_model():
    x_train, y_train, x_test, y_test, n_features = datasets()
    x_train, y_train = shuffle(x_train, y_train)

    model = Sequential()
    model.add(Dense(128, input_dim=n_features, activation='relu', kernel_regularizer='l2'))
    model.add(Dropout(0.3))

    model.add(Dense(64, activation='relu', kernel_regularizer='l2'))
    model.add(Dropout(0.4))

    model.add(Dense(32, activation='relu', kernel_regularizer='l2'))
    model.add(Dropout(0.5))

    model.add(Dense(1, activation='sigmoid'))

    print(model.summary())
    lr_schedule = ExponentialDecay(initial_learning_rate=1e-3, decay_steps=5000, decay_rate=0.9)
    optimizer = Adam(learning_rate=lr_schedule)

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

    history = model.fit(
        x_train, y_train,
        validation_split=0.2,
        batch_size=64,
        epochs=50,
        callbacks=[early_stop, reduce_lr],
        verbose=2
    )

    _, train_acc = model.evaluate(x_train, y_train, verbose=2)
    _, test_acc = model.evaluate(x_test, y_test, verbose=2)
    print(f'Train: {train_acc:.3f}, Test: {test_acc:.3f}')

    y_class_pred = (model.predict(x_test) > 0.5).astype("int32")
    y_pred = model.predict(x_test)
    graph_acc(history)
    matrix = conf_matrix(y_test, y_class_pred)
    prediction_metrics(y_test, y_pred, y_class_pred, matrix)

    return model


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import snntorch as snn
import snntorch.utils as utils
from torch.utils.data import DataLoader, TensorDataset

class SpikingCNN(nn.Module):
    def __init__(self, input_shape):
        super(SpikingCNN, self).__init__()

        # Define network layers
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * (input_shape[1] // 8), 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

        # Spiking neural network components
        self.lif1 = snn.Leaky(beta=0.9, spike_grad=snntorch.gradiret.corrd, mem_update=True)
        self.lif2 = snn.Leaky(beta=0.9, spike_grad=snntorch.gradiret.corrd, mem_update=True)
        self.lif3 = snn.Leaky(beta=0.9, spike_grad=snntorch.gradiret.corrd, mem_update=True)

    def forward(self, x):
        # Convolutional layers with spiking neurons
        x = self.conv1(x)
        x, _ = self.lif1(x)
        x = self.pool(x)

        x = self.conv2(x)
        x, _ = self.lif2(x)
        x = self.pool(x)

        x = self.conv3(x)
        x, _ = self.lif3(x)
        x = self.pool(x)

        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)

        return x

def train_snn(model, train_loader, optimizer, criterion, device):
    model.train()
    losses = []
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
    return losses

def evaluate_snn(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    all_targets = []
    all_predictions = []
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            predicted = (outputs > 0.5).int()
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
            all_targets.extend(targets.cpu().numpy())
            all_predictions.extend(outputs.cpu().numpy())
    accuracy = correct / total
    return accuracy, np.array(all_targets), np.array(all_predictions)



In [ ]:
def main():
    x_train, y_train, x_test, y_test, n_features = datasets()
    x_train, x_test = x_train.reshape(-1, 1, n_features).astype(np.float32), x_test.reshape(-1, 1, n_features).astype(np.float32)
    y_train, y_test = y_train.astype(np.float32), y_test.astype(np.float32)

    train_dataset = TensorDataset(torch.tensor(x_train), torch.tensor(y_train))
    test_dataset = TensorDataset(torch.tensor(x_test), torch.tensor(y_test))

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = SpikingCNN(input_shape=(1, n_features)).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.BCEWithLogitsLoss()

    train_losses = train_snn(model, train_loader, optimizer, criterion, device)
    accuracy, y_test, y_pred_prob = evaluate_snn(model, test_loader, device)

    y_pred = (y_pred_prob > 0.5).astype(int)

    print(f'Accuracy: {accuracy:.3f}')

    plot_metrics(history, train_losses)
    matrix = conf_matrix(y_test, y_pred)
    prediction_metrics(y_test, y_pred, matrix)

if __name__ == "__main__":
    main()

In [ ]:
def plot_metrics(history, train_losses, val_losses):
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Training Loss')
    plt.title('Training Loss vs Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Plot ROC curve
    plt.subplot(1, 2, 2)
    fpr, tpr, _ = roc_curve(y_test_bin, y_pred_prob)
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % auc(fpr, tpr))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.title('ROC Curve')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend()
    plt.show()

In [ ]:
def conf_matrix(y_test, y_pred):
    matrix = confusion_matrix(y_test, y_pred)
    df_cm = pd.DataFrame(matrix, columns=[0, 1], index=[0, 1])
    df_cm.index.name = 'Truth'
    df_cm.columns.name = 'Predicted'
    plt.figure(figsize=(10, 7))
    sns.heatmap(df_cm, cmap="BuGn", annot=True, annot_kws={"size": 16})
    plt.title('Confusion Matrix')
    plt.show()
    return matrix